# Using XeSim

## Imports

In [1]:
import strax
import cutax
import XeSim

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

*** Detector definition message ***
You are currently using the default XENON10 template detector.



Using nestpy version 1.5.4


## Microphysics simulation (former epix)

In XeSim all simulation steps are handled in dedicated plugins just like straxen does for our data processing. The first part of our full chain simulations is the simulation of microphysics effects. The corresponding plugins are grouped in the `micro_physics` module.

To set up a simulation we first need to define a `Context`. At this stage we dont't need a lot of inputs so we can run the simulation inside a generic `strax.Context`. Make sure to register all needed plugins and define an `DataDirectory`. Afterwards we can change the configuration of the simulation using `st.set_config`. If you want to try your own `.root` or `.csv`file feel free to change the given example file.

There are some more options we can change: 
- `debug` will print out some debug informations during the simulation (For now there are basically no debug statements.)
- `source_rate`: You can specify a rate in Hz. XeSim will distribute your events according to this rate. 
- `n_interactions_per_chunk`: XeSim builds strax chunks based on an algorithm called 'dynamic chunking'. It searches for empty time intervalls and cuts the data into chunks there. With this parameter you can set an lower limit to the number of interactions per chunk. For Geant4 files something in the order of 10k - 100k can work. For csv files i would reccomend something in the order of 10 - 1000 depending on you specific data. 
- `cut_delayed`: XeSim will remove interactions that happen after the last sampled time + the cut_delayed time. We dont want to deal with interactions that happen way after our "run" finished (yet)
- `DetectorConfigOverride`: Part of the epix detector config - we will update this part at some point. (Here you can set your electric field for the microphysics simulation)

In [2]:
st = strax.Context(register = [XeSim.micro_physics.ChunkInput,
                               XeSim.micro_physics.FindCluster,
                               XeSim.micro_physics.MergeCluster,
                               XeSim.micro_physics.ElectricField,
                               XeSim.micro_physics.NestYields,
                               #XeSim.micro_physics.BBFYields,
                               XeSim.micro_physics.MicroPhysicsSummary],
                   storage = [strax.DataDirectory('/scratch/midway2/hschulze/epix_data/')]
                  )

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "debug": True,
               "source_rate": 1,
               #"n_interactions_per_chunk": 100,
               "cut_delayed": 4e14,
               "DetectorConfigOverride" : "sr0_epix_detectorconfig.ini",
              })

Now that we have our context we can start to run XeSim. Just use `st.make`. Probably there will be a lot of warnings...

In [3]:
run_number = "00000"

st.make(run_number, "geant4_interactions")
st.make(run_number, "cluster_index")
st.make(run_number, "clustered_interactions")
st.make(run_number, "electric_field_values")
st.make(run_number, "quanta")
st.make(run_number, "microphysics_summary")

If your simulation did not crash you can now load the simulated data just like straxen data. First we can take a look at the `geant4_interaction`. Each line represents a single energy deposit. 

In [4]:
geant4_interactions = st.get_df("00000",["geant4_interactions", "cluster_index"])
geant4_interactions.head(10)

Loading plugins: |          | 0.00 % [00:00<?]

,cluster_ids,time,endtime,x,y,z,t,ed,type,trackid,parenttype,parentid,creaproc,edproc,evtid,x_pri,y_pri,z_pri
0,0,1125955395,1125955395,-20.890713,-51.367107,-1.464800,21.859256,5.101432,Xe131,2,neutron,1,hadElastic,ionIoni,0,-49.813919,0.516251,9.241811
1,0,1234826432,1234826432,-25.334255,-4.493484,-21.541744,8.586755,2.472048,Xe131,102,neutron,2,hadElastic,ionIoni,1,-44.154747,2.803934,7.933488
2,1,1234826432,1234826432,-25.548460,-4.034955,-21.613764,8.223299,72.487770,Xe129,3,neutron,1,neutronIne,ionIoni,1,-44.154747,2.803934,7.933488
3,2,1234826433,1234826433,-10.058850,14.729630,-38.732780,9.384739,0.036780,e-,93,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
4,2,1234826433,1234826433,-10.058850,14.729630,-38.732780,9.384740,0.022620,e-,94,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
5,2,1234826433,1234826433,-10.058849,14.729630,-38.732780,9.384740,0.012420,e-,95,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
6,2,1234826433,1234826433,-10.058850,14.729630,-38.732780,9.384740,0.033880,e-,96,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
7,2,1234826433,1234826433,-10.058849,14.729630,-38.732780,9.384739,0.012420,e-,97,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
8,2,1234826433,1234826433,-10.058849,14.729631,-38.732780,9.384739,0.044470,e-,98,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
9,2,1234826433,1234826433,-10.058849,14.729630,-38.732780,9.384739,0.044470,e-,99,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488


You can also load the clustered data that will go into the second part of the simulation. For now it is called `microphysics_summary`. Each line gives you now the position and number of electron and photons (along other informations.)

In [5]:
microphysics_summary = st.get_df("00000",[ "microphysics_summary"])
microphysics_summary.head(10)

Loading ['microphysics_summary']: |          | 0.00 % [00:00<?]

,x,y,z,ed,nestid,A,Z,evtid,x_pri,y_pri,z_pri,xe_density,vol_id,create_S2,time,endtime,e_field,photons,electrons,excitons
0,-20.890713,-51.367107,-1.464800,5.101432,0,0,0,0,-49.813919,0.516251,9.241811,2.862,1,True,1125955395,1125955395,35,34.0,31.0,27.0
1,-25.334255,-4.493484,-21.541744,2.472048,0,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826432,1234826432,25,18.0,13.0,16.0
2,-25.548460,-4.034955,-21.613764,72.487770,0,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826432,1234826432,25,1068.0,171.0,629.0
3,-10.058967,14.729661,-38.732719,29.781300,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,1895.0,312.0,348.0
4,-10.057805,14.731761,-38.735424,4.774700,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,205.0,141.0,12.0
5,-10.056870,14.731192,-38.734726,4.317931,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,214.0,92.0,11.0
6,-10.055875,14.729283,-38.733784,28.615028,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,1734.0,386.0,284.0
7,-10.055321,14.730186,-38.735222,24.771402,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,1438.0,377.0,259.0
8,-10.054443,14.729301,-38.734734,32.040482,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,1838.0,545.0,357.0
9,-10.054362,14.728804,-38.735455,86.527084,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1234826433,1234826433,23,5048.0,1310.0,954.0


## Detector and Electronics Simulation (former WFSim)

Now that you know how to run microphysics simulations we can continue with the detector physics, PMT and DAQ simulations. The corresponding plugins can be found in `detector_physics` and `pmt_and_daq`. 

In [6]:
full_chain_modules = [XeSim.micro_physics.ChunkInput,
                      XeSim.micro_physics.FindCluster,
                      XeSim.micro_physics.MergeCluster,
                      XeSim.micro_physics.ElectricField,
                      XeSim.micro_physics.NestYields,
                      XeSim.micro_physics.MicroPhysicsSummary,
                      XeSim.detector_physics.S1PhotonPropagation,
                      XeSim.detector_physics.ElectronDrift,
                      XeSim.detector_physics.ElectronExtraction,
                      XeSim.detector_physics.ElectronTiming,
                      XeSim.detector_physics.SecondaryScintillation,
                      XeSim.detector_physics.S2PhotonPropagation,
                      XeSim.pmt_and_daq.PMTAfterPulses,
                      XeSim.pmt_and_daq.PMTResponseAndDAQ,
                     ]

We want to simulate now up to the raw_records level and then process the data with our usual straxen processing routines. To do so, we will now hijack the simulation context from cutax and register our modules. In the future we aim for a dedicated context where we don't need to do this manualy. Remember to select an output folder with sufficient free disk space as raw_records data can be a bit heavy. 

In [7]:
st = cutax.contexts.xenonnt_sim_SR0v3_cmt_v9(output_folder = "/scratch/midway2/hschulze/RefactorTest")

for module in full_chain_modules:
    st.register(module)

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "debug": True,
               "source_rate": 1,
               #"n_interactions_per_chunk": 100,
               "cut_delayed": 4e14,
               "DetectorConfigOverride" : "sr0_epix_detectorconfig.ini",
              })

/dali/lgrandi/xenonnt/software/cutax/v1.14.3/cutax/contexts.py:123: UserWarning: WARNING! You are using SR0_V3 simulation context! Consider updating to a newer/pinned version (SR0_V4)!
  warnings.warn(f'WARNING! You are using {faxconf_version.upper()} simulation context!'
/dali/lgrandi/xenonnt/software/cutax/v1.14.3/cutax/contexts.py:148: UserWarning: WARNING! [ FAX_CONFIG_NT_SR0_V3.JSON ] does not contain "field_distortion_correction_map" key!
  warnings.warn(f'WARNING! [ {fax_conf.upper()} ] does not contain "{fax_config_fdc_key}" key!')


### Microphysics
Just like before we first run the microphysics simulation. Lets make `microphysics_summary` first. 

In [8]:
run_number = "00000"

st.make(run_number,"microphysics_summary")

### S1 Simulation

After the microphysics simulation is done we can continue and distribute our S1 photons to our PMTs. The output will be a long list where each row represents a single photon with the information of `time` and `channel`attached.  Strax can show you a progress bar during simulation. I found it to be a bit buggy from times in combination with XeSim. 

In [9]:
st.make(run_number,"S1_channel_and_timings" ,progress_bar = True)

### S2 Simulations

The S2 simulations are a bit more complicated than the S1 simulation. We first need to drift our electrons to the liquid-gas interface, extract them and calculate the timing of the electrons when reaching the gas phase. Afterwards we can simulate how many photons each electron generates and then distribute the photons on the PMTs. The output of the S2 simulation has basically the same format as the S1 simulation output. 

In [10]:
st.make(run_number,"drifted_electrons",progress_bar = True)
st.make(run_number,"extracted_electrons",progress_bar = True)
st.make(run_number,"electron_time",progress_bar = True)
st.make(run_number,"photons",progress_bar = True)
st.make(run_number,"sum_photons",progress_bar = True)
st.make(run_number,"photon_channels_and_timeing",progress_bar = True)

### PMT Afterpulses

Based on the output of the S1 and S2 simulation we can calculate PMT afterpulses. The output will have the same format as the S1 and S2 simulations where each row represents a "virtual" photon with information about channel and timing attached. 

In [11]:
st.make(run_number,"pmt_afterpulses",progress_bar = True)

### PMT and DAQ

Combining our S1, S2 and AP simulations we can start to simulate the PMT response to photons and simulate the effects of our DAQ on the PMT output. As a result we get `raw_records` that sould look very similar to the data we get from our real TPC and DAQ! 

In [12]:
st.make(run_number,"raw_records" , progress_bar = True)

## Processing

We finished our simulation! Now we can use straxen to process it. All necessary straxen plugins should be already registered in the context. 

In [13]:
st.make(run_number,"event_info")

In [14]:
event_info = st.get_df(run_number, "event_info")

Loading event_info: |          | 0.00 % [00:00<?]

In [15]:
event_info.head(20)

,time,endtime,cs1,cs1_wo_timecorr,cs2_wo_elifecorr,cs2_wo_timecorr,cs2_area_fraction_top,cs2_bottom,cs2,alt_cs1,...,alt_s2_z_naive,r_field_distortion_correction,alt_s1_r_field_distortion_correction,alt_s2_r_field_distortion_correction,z_field_distortion_correction,alt_s1_z_field_distortion_correction,alt_s2_z_field_distortion_correction,alt_s1_theta,alt_s2_theta,theta
0,1661918000,1663191320,7.401153,7.401153,6.502084e+02,6.236501e+02,0.729429,176.186157,6.511631e+02,NaN,...,NaN,1.504706,NaN,NaN,0.000000,NaN,NaN,-1.941818,NaN,-1.941818
1,1872622606,1875845730,67401.117188,67401.117188,1.361428e+06,1.354244e+06,0.729146,382984.968750,1.413988e+06,NaN,...,-8.731192,2.720521,NaN,4.092588,0.097754,NaN,1.018577,2.205207,-2.612120,2.205207
2,5143003086,5145927560,16412.974609,16412.974609,1.383825e+03,1.405367e+03,0.700990,438.757324,1.467366e+03,NaN,...,-45.747852,6.606839,NaN,6.832341,0.372208,NaN,0.513075,-1.883837,-1.896925,-1.883837
3,5506851916,5509836960,28373.750000,28373.750000,2.452191e+05,2.554548e+05,0.742079,64377.234375,2.496007e+05,NaN,...,-16.097536,5.888733,NaN,2.186403,1.016815,NaN,0.149172,2.745961,2.789273,2.745961
4,5878034746,5881143160,119160.523438,119160.523438,9.460058e+05,9.223637e+05,0.738890,251463.453125,9.630547e+05,NaN,...,-26.867632,-1.272949,NaN,2.673297,0.045800,NaN,0.133326,-1.221478,-1.115638,-1.221478
5,6099036256,6102465020,5223.350098,5223.350098,1.646648e+05,1.963338e+05,0.743433,49218.296875,1.918345e+05,4251.065918,...,-108.186836,8.754766,8.756236,6.889683,0.249465,0.253260,0.219601,2.783427,1.550198,2.783427
6,7211794666,7215521350,26028.996094,26028.996094,4.991675e+05,4.795186e+05,0.727356,136505.656250,5.006730e+05,NaN,...,-68.235207,-1.050691,NaN,1.568062,0.209953,NaN,0.018019,-2.581830,0.769916,-2.581830
7,7952145436,7954863700,13355.971680,13355.971680,3.260337e+05,3.374955e+05,0.735257,87302.062500,3.297612e+05,NaN,...,NaN,4.193702,NaN,NaN,0.817684,NaN,NaN,-1.250110,NaN,-1.250110
8,9419389776,9422098180,46.120838,46.120838,2.140263e+03,2.191868e+03,0.757613,519.105164,2.141638e+03,NaN,...,NaN,2.488417,NaN,NaN,0.000000,NaN,NaN,-2.782295,NaN,-2.782295
9,10339977256,10344149040,99128.187500,99128.187500,2.593342e+06,2.718442e+06,0.733706,755840.187500,2.838369e+06,NaN,...,-95.358803,1.720505,NaN,0.267624,0.016310,NaN,0.000373,-2.547947,-2.568428,-2.547947
